Імпорти

In [1]:
import pandas as pd
import urllib.request
from datetime import datetime
from os import listdir
from datetime import datetime, date
import csv

Змінні

In [2]:
DIRECTORY = 'CSV_Files/'
URL = ('https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={'
       '}&year1=1981&year2=2020&type=Mean')
INDEXES = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13,
           17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 24: 1, 25: 2, 26: 7, 27: 5, 28:6}

Завантаження файлів

In [3]:
for province_id in range(1, 29):
    url = URL.format(province_id)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.now()
    date_and_time_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open(DIRECTORY + 'NOAA_ID' + str(province_id) + '_' + date_and_time_time + '.csv', 'wb')
    out.write(text)
    out.close()
    # print('File for ' + str(province_id))
print('Success')

Success


Створення фрейму з файлів у папці

In [4]:

res = []

for i in range(1,29):
    date_of_file = datetime(1970, 1, 1, 22, 22, 22)
    file = ""
    for file_path in listdir(DIRECTORY):
        if file_path.endswith('.csv') and "ID" + str(i) in file_path:
            file_datetime = file_path.split("_")[2] + "_" + file_path.split("_")[3][:8]
            format_str = "%d-%m-%Y_%H-%M-%S"
            obj_date = datetime.strptime(file_datetime, format_str)
            if obj_date > date_of_file:
                file = file_path
                date_of_file = obj_date
    res.append(file)
print(res)

headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
df_list = []

for filename in res:
    df = pd.read_csv(DIRECTORY + filename, header=1, names=headers)
    id = filename.split("_")[1][2:]
    df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
    df = df.drop(df.index[-1])
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df = df.drop('empty', axis=1)
    df['area'] = int(id)
    df_list.append(df)
df = pd.concat(df_list)

['NOAA_ID19_11-03-2024_18-23-00.csv', 'NOAA_ID28_11-03-2024_18-23-07.csv', 'NOAA_ID3_11-03-2024_18-22-44.csv', 'NOAA_ID4_11-03-2024_18-22-46.csv', 'NOAA_ID5_11-03-2024_18-22-48.csv', 'NOAA_ID6_11-03-2024_18-22-48.csv', 'NOAA_ID7_11-03-2024_18-22-49.csv', 'NOAA_ID8_11-03-2024_18-22-50.csv', 'NOAA_ID9_11-03-2024_18-22-51.csv', 'NOAA_ID10_11-03-2024_18-22-52.csv', 'NOAA_ID11_11-03-2024_18-22-53.csv', 'NOAA_ID12_11-03-2024_18-22-54.csv', 'NOAA_ID13_11-03-2024_18-22-55.csv', 'NOAA_ID14_11-03-2024_18-22-55.csv', 'NOAA_ID15_11-03-2024_18-22-56.csv', 'NOAA_ID16_11-03-2024_18-22-57.csv', 'NOAA_ID17_11-03-2024_18-22-58.csv', 'NOAA_ID18_11-03-2024_18-22-59.csv', 'NOAA_ID19_11-03-2024_18-23-00.csv', 'NOAA_ID20_11-03-2024_18-23-00.csv', 'NOAA_ID21_11-03-2024_18-23-01.csv', 'NOAA_ID22_11-03-2024_18-23-02.csv', 'NOAA_ID23_11-03-2024_18-23-03.csv', 'NOAA_ID24_11-03-2024_18-23-04.csv', 'NOAA_ID25_11-03-2024_18-23-05.csv', 'NOAA_ID26_11-03-2024_18-23-05.csv', 'NOAA_ID27_11-03-2024_18-23-06.csv', 'NOAA_I

Зміна індексів

In [5]:
df.replace({'area': INDEXES}, inplace=True)

Перша процедура

In [6]:
def procedure_1(df, year, province_n):
    df_new = df[(df.area == province_n) & (df.Year.astype(str) == str(year))]['VHI']
    print('For ' + str(year) + ' and the province ' + str(province_n) + ':')
    print('Min: ' + str(df_new.min()))
    print('Max: ' + str(df_new.max()))

Друга процедура

In [7]:
def procedure_2(df, province_n, percent=50):
    df_drought = df[(df["area"] == province_n) & (df.VHI <= percent)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' extreme drought is in:')
    print('Years: ' + ' '.join(years))

Третя процедура

In [8]:
def procedure_3(df, province_n):
    df_drought = df[(df["area"] == province_n) & (df.VHI <= 35) & (df.VHI > 15)]
    list_from_ds = list(df_drought.Year)
    years = []
    [years.append(item) for item in list_from_ds if item not in years]
    print('For the province ' + str(province_n) + ' moderate drought is in:')
    print('Years: ' + ' '.join(years))

Завантаження всіх файлів

In [9]:
for i in range(1, 20):
    procedure_1(df, 1999, i)

For 1999 and the province 1:
Min: 22.25
Max: 62.36
For 1999 and the province 2:
Min: 22.41
Max: 64.03
For 1999 and the province 3:
Min: 34.45
Max: 66.66
For 1999 and the province 4:
Min: 21.56
Max: 58.85
For 1999 and the province 5:
Min: 23.34
Max: 69.05
For 1999 and the province 6:
Min: 25.61
Max: 62.67
For 1999 and the province 7:
Min: 31.99
Max: 69.05
For 1999 and the province 8:
Min: 36.98
Max: 66.02
For 1999 and the province 9:
Min: 21.31
Max: 67.42
For 1999 and the province 10:
Min: 33.23
Max: 70.57
For 1999 and the province 11:
Min: 33.59
Max: 57.32
For 1999 and the province 12:
Min: 12.88
Max: 66.08
For 1999 and the province 13:
Min: 36.31
Max: 80.35
For 1999 and the province 14:
Min: 29.32
Max: 76.21
For 1999 and the province 15:
Min: 29.94
Max: 63.09
For 1999 and the province 16:
Min: 22.27
Max: 67.37
For 1999 and the province 17:
Min: 24.89
Max: 69.55
For 1999 and the province 18:
Min: 36.23
Max: 72.98
For 1999 and the province 19:
Min: 27.63
Max: 62.69


In [10]:
procedure_2(df, 3)

For the province 3 extreme drought is in:
Years: 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020


In [11]:
procedure_3(df, 3)

For the province 3 moderate drought is in:
Years: 1982 1983 1984 1985 1986 1988 1989 1990 1992 1993 1994 1995 1996 1997 1999 2000 2001 2002 2003 2005 2006 2007 2008 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020
